<a href="https://colab.research.google.com/github/datle2403/datle2403/blob/main/Adaline_advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# shuffle the data
# sgd adaline
# fit_partial: not reinitialized weight 

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
class adaline(BaseEstimator, TransformerMixin):
  def __init__(self,iter=50,lr=0.001,random_state=1,shuffle=True):
    self.shuffle=shuffle
    self.iter=iter
    self.lr=lr
    self.random_state=random_state
    self.initialize_weight=False
  def net_input(self,X):
    return self.w_[0] + np.dot(X,self.w_[1:])
  def linear_activation_function(self,z):
    return z
  def fit(self,X,y):
    self.error_=[]
    self.create_weight(X.shape[1])
    for _ in range(self.iter):
      cost=[]
      if self.shuffle:
        self._shuffle(X,y)
      for x,y_target in zip(X,y):
        cost.append(self.update_weight(x,y_target))
      avg_cost=sum(cost)/len(y)
      self.error_.append(avg_cost)
    return self
  def predict(self,X):
    return np.where(self.net_input(X)>0.0,1,-1)
  def cost_function(self,X,y):
    return 1/2*np.sum((y-self.net_input(X))**2)
  def create_weight(self,m):
    self.rgen = np.random.RandomState(self.random_state)
    self.w_ = self.rgen.normal(loc=0.0, scale=0.01, size=1 + m)
    self.initialize_weight=True
  def _shuffle(self,X,y):
    r=self.rgen.permutation(len(y))
    return X[r],y[r]
  def update_weight(self,xi,target):
    Z=self.net_input(xi)
    A=self.linear_activation_function(Z)
    update=self.lr*(target-A)
    self.w_[0]+=np.sum(update)
    self.w_[1:]+=np.dot(xi.T,update)
    error=self.cost_function(xi,target)
    return error
  def partial_fit(self,X,y):
    """Fit training data without reinitializing the weights"""
    if not self.initialize_weight:
      self.create_weight(X.shape[1])
    if y.ravel().shape[0]>1:
      for xi, target in zip(X, y):
            self.update_weights(xi, target)
    else:
      self.update_weight(X,y)
    return self


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
#load dataset
df=pd.read_csv('Iris.csv')
X=df.iloc[:100,[1,3]].values
y=df.iloc[:100,5].values
y=np.where(y=='Iris-setosa',-1,1)
X_std = np.copy(X)
X_std[:,0] = (X[:,0] - X[:,0].mean()) / X[:,0].std()
X_std[:,1] = (X[:,1] - X[:,1].mean()) / X[:,1].std()
ada_sgd = adaline(iter=15, lr=0.01, random_state=1)
ada_sgd.fit(X_std, y)
ada_sgd.predict(X_std)
ada_sgd.error_

[0.19447020651917138,
 0.06839371835284758,
 0.05263515485568057,
 0.043532448191064575,
 0.037256205344234974,
 0.0328966609148155,
 0.02989255077961617,
 0.027837008993548063,
 0.026437468957432233,
 0.025488197386452374,
 0.024846582556764116,
 0.024414537715050488,
 0.02412488869484881,
 0.023931751967044584,
 0.023803846949539118]

In [ ]:
#add grid search
from sklearn.model_selection import GridSearchCV
iter=[x for x in range(0,100)]
param_grid = [
    {'iter': iter},
    {'lr':[0.1,0.01,0.001,0.0001]}
  ]
grid_search = GridSearchCV(ada_sgd, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True,refit=True)
grid_search.fit(X_std,y)

GridSearchCV(cv=5, estimator=adaline(iter=15, lr=0.01),
             param_grid=[{'iter': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                   14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
                                   25, 26, 27, 28, 29, ...]},
                         {'lr': [0.1, 0.01, 0.001, 0.0001]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [ ]:
grid_search.best_params_

{'iter': 8}

adaline(iter=8, lr=0.01)